In [4]:
import openai
import os
import numpy as np
import time
import re
import pickle
import json

In [5]:
# Ścieżka do folderu z plikami
dataset_path = "../../ML_Model/datasets/tested_dataset/final_dataset/final_test.pkl"

# Preprocessing dataset

In [6]:
# Wczytanie pliku
with open(dataset_path, 'rb') as f:
    data = pickle.load(f)

In [7]:
labels = data['labels']
answers = data['answer']
evaluations = data['rating']
fps = data['fps']
filenames = data['video_url']
tricks = data['tricks']
labels_type = [
    "Squat-depth" ,
    "Back-round" ,
    "Taking-off-foot",
    "Knee-collapse" ,
    "Butt-wink" ,
    "Dominant-hip" ,
    "Lack-of-abdominal-tension",
    "No-knee-outlet" 
]

# Api

In [8]:
# Inicjalizacja klienta Cohere
# https://cohere.com/

with open('../api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'gpt:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('gpt:')[1].strip()

# Moje api
openai.api_key = key_api

from openai import OpenAI

client = OpenAI(
    api_key = key_api,
)

# Generowanie opdowiedzi

In [9]:
def query_gpt(prompt, data):
    try:
        prompt += "Data: " + data[0]
        #prompt += f"\n Summary of technique:  {data[1]}"
        prompt += f"\n Overall evaluation:  {data[2]}"
        #prompt += f"\n fps of video: {data[3]}"
        
        # Query GPT using the new API method
        response =  client.chat.completions.create(
            model="gpt-3.5-turbo",  # You can use "gpt-4", "gpt-4-turbo", or "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=50,
            temperature=0.7
        )
        
        # Extract and return the response text
        return response.choices[0].message.content.strip()

    except Exception as e:
        print("error")
        return str(e)

In [10]:
# Zbudowanie promptu na podstawie komentarzy
prompt = "You will get data about a recording in which there is a person squatting with a barbell and weight. Then you will get data for this recording, the first is an array where in each row there is an error and information about the time intervals in which the error occurred. Analyze this data and describe it for the user. Do not describe when the errors occurred, but rather what errors occurred and how many times. In the second section describe how the problems can be solved. List possible causes of their occurrence. Do not exceed 1000 characters. Avoid general answers like: 'The error is due to poor technique' "

In [11]:
analysis_comments_prompt = {}
# Ustawienie wyświetlania pełnych tablic
np.set_printoptions(threshold=np.inf)

# Wyświetlenie wyników
for i, filename in enumerate(filenames):
    start_time = time.time()

    print(f"Wyniki dla nagrania: {filename}, numer {i}")
    if i >= 4:
        break
        
    label = labels[i]
    answer = answers[i]
    evaluation = json.loads(evaluations[i])[0]["number"]
    fps_ = fps[i]
    trick = tricks[i]
    
    ## Łączenie labels z labels_type
    #combined_data = []
    #for label_row, label_type in zip(trick, labels_type):
    #    combined_data.append({"type": label_type, "values": label_row})
        
    result_string = ""
    if not isinstance(trick, float):
        for item in trick:
            result_string += f"start: {round(item['start'],2)}, end: {round(item['end'],2)} : {item['labels']}\n"
            
    data_input = [result_string, answer, evaluation, fps_]
    
    # Generowanie analizy techniki, jeśli komentarze istnieją
    analysis = query_gpt(prompt, data_input)

    # Dodajemy wynik do listy
    analysis_comments_prompt[filename] = {
        'Analysis': analysis
    }
    
    print('*'*100)
    print(f"Analysis: \n {analysis}\n")
    print('*'*100)
    
    end_time = time.time()
    print(f"Przetwarzanie trwało {end_time - start_time:.4f} sekundy")
    
    print("-" * 40)

Wyniki dla nagrania: s3://squat-label-studio/squats/lifting/0-0084-50-7-0-2-0-0.mp4, numer 0
****************************************************************************************************
Analysis: 
 From the data provided, it appears that there were two instances of errors observed during the recording of a person squatting with a barbell and weight. The errors identified were related to the "Dominant-hip" and occurred during two time intervals:

****************************************************************************************************
Przetwarzanie trwało 1.3112 sekundy
----------------------------------------
Wyniki dla nagrania: s3://squat-label-studio/squats/lifting/0-0085-80-8-1-2-0-0.mp4, numer 1
****************************************************************************************************
Analysis: 
 In the recording, errors observed include: 1) Excessive forward lean during squat, occurring 3 times. 2) Barbell path deviation, occurring 2 times. 3) Weight 